# Homework Agents - LLM ZOOMCAMP - Rui Pinto

In this homework, we will learn more about function calling,
and we will also explore MCP - model-context protocol.

## Preparation

First, we'll define a function that we will use when building
our agent. 

It will generate fake weather data:

```python
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)
```

In [1]:
import random


def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


known_weather_data = {"berlin": 20.0}

In [2]:
print(get_weather("Berlin"))  # Example usage, can be removed or modified as needed
print(
    get_weather("London")
)  # we do not have data for London, so it will return a random temperature

20.0
26.6


## Q1. Define function description

We want to use it as a tool for our agent, so we need to 
describe it 

How should the description for this function look like? Fill in missing parts

```python
get_weather_tool = {
    "type": "function",
    "name": "<TODO1>",
    "description": "<TODO2>",
    "parameters": {
        "type": "object",
        "properties": {
            "<TODO3>": {
                "type": "string",
                "description": "<TODO4>"
            }
        },
        "required": [TODO5],
        "additionalProperties": False
    }
}

### tool to get the weather for a city

In [3]:
# TODO1 - get_weather
# TODO2 - get the current weather for a specific city
# TODO3 - city
# TODO4 - the name of the city to get the weather for
# TODO5 - "city"

get_weather_tool = {
    "type": "function",
    "name": "get_weather",  # Function to get the current temperature for a city
    "description": "Get current temperature for provided city in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The name of the city to get the weather temperature for.",
            }
        },
        "required": ["location"],
        "additionalProperties": False,
    },
}

What did you put in `TODO3`?

- location ✅

## Testing it (Optional)

If you have OpenAI API Key (or alternative provider),
let's test it.

A question could be "What's the weather like in Germany?"

Experiment with different system prompts to have better answers
from the system.

You can use [chat_assistant.py](https://github.com/alexeygrigorev/rag-agents-workshop/blob/main/chat_assistant.py)
or implement everything yourself 

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
```

In [4]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

### OpenAI chat assistance calling

In [5]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

input_message = {
    "role": "user",
    "content": "What's the weather like in Germany?",
}

response = client.responses.create(
    model="gpt-4o-mini", input=[input_message], tools=[get_weather_tool]
)

print(response.output)

[ResponseFunctionToolCall(arguments='{"location":"Germany"}', call_id='call_nTRiCzTh0CQq9gMMMXq3r2CB', name='get_weather', type='function_call', id='fc_687e5a6cfc308199a2f6ef1e30fdfc3f0dc1c4cf1f397684', status='completed')]


In [6]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

# If your tool uses "location" as the key:
city = args["location"]
weather = get_weather(city)
print(f"The weather in {city} is {weather} °C")  # fake weather data

The weather in Germany is 27.2 °C


## Q2. Adding another tool

Let's add another tool - a function that can add weather data
to our database:

```python
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'
```

Now let's write a description for it.

What did you write?

Optionally, you can test it after adding this function.

In [7]:
def set_weather(location: str, temp: float) -> None:
    location = location.strip().lower()
    known_weather_data[location] = temp
    return "OK"

### tool to set the weather for a city

In [8]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set the current temperature for a specific location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The name of the location to set the weather for.",
            },
            "temp": {
                "type": "number",
                "description": "The temperature to set for the location (in Celsius).",
            },
        },
        "required": ["location", "temp"],
        "additionalProperties": False,
    },
}

In [9]:
# testing

import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

input_message = {
    "role": "user",
    "content": "Set the weather for Paris to 25 °C?",
}

response = client.responses.create(
    model="gpt-4o-mini", input=[input_message], tools=[set_weather_tool]
)

print(response.output)

[ResponseFunctionToolCall(arguments='{"location":"Paris","temp":25}', call_id='call_teosXggyAfi4B9HAUYUfxjRE', name='set_weather', type='function_call', id='fc_687e5a6e4a74819ab226f7673cac462008bb74cfdf668614', status='completed')]


In [10]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

# If your tool uses "location" as the key:
location = args["location"]
confirmation = set_weather(location, args["temp"])
print(
    f"Set the weather in {location} was successful: {confirmation} to {args['temp']} °C"
)

Set the weather in Paris was successful: OK to 25 °C


### Multiple tools

In [11]:
# adding more tools
tools = [get_weather_tool, set_weather_tool]

In [12]:
def do_call(entry):
    if entry.name == "get_weather":
        args = json.loads(entry.arguments)
        return get_weather(args["location"])
    elif entry.name == "set_weather":
        args = json.loads(entry.arguments)
        return set_weather(args["location"], args["temp"])
    else:
        raise ValueError(f"Unknown tool call: {entry.name}")


In [17]:
# Store the chat history
chat_messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. After each function call, use the result to answer the user's question directly.",
    }
]

max_steps = 5
steps = 0

# Main agent loop
while steps < max_steps:
    # Get user input
    question = input("Ask a question (or type 'stop' to exit): ")
    if question.lower() == "stop":
        break

    # Add user message to chat history
    chat_messages.append({"role": "user", "content": question})

    # Keep looping until we get a final assistant message
    while True:
        # Send chat history and tools to OpenAI
        response = client.responses.create(
            model="gpt-4o-mini", input=chat_messages, tools=tools
        )

        has_function_call = False

        # Process each entry in the response
        for entry in response.output:
            if entry.type == "function_call":
                print(
                    f"Function call requested: {entry.name} with arguments {entry.arguments}"
                )
                result = do_call(entry)
                args = json.loads(entry.arguments)
                if entry.name == "get_weather":
                    chat_messages.append(
                        {
                            "role": "assistant",
                            "content": f"The weather in {args['location'].title()} is {result} °C.",
                        }
                    )
                elif entry.name == "set_weather":
                    chat_messages.append(
                        {
                            "role": "assistant",
                            "content": f"Set the weather in {args['location'].title()} to {args['temp']} °C: {result}",
                        }
                    )
                has_function_call = True
                break
            elif entry.type == "message":
                print("Assistant:", entry.content)
                chat_messages.append({"role": "assistant", "content": entry.content})
                has_function_call = False
                break

        if has_function_call:
            continue  # Make a new API call with the function output
        else:
            break  # End the loop after printing the assistant's message
    steps += 1

Function call requested: get_weather with arguments {"location":"Berlin"}
Function call requested: get_weather with arguments {"location":"Berlin"}
Function call requested: get_weather with arguments {"location":"Berlin"}
Assistant: [ResponseOutputText(annotations=[], text='The weather in Berlin is currently 25 °C.', type='output_text', logprobs=[])]
Function call requested: get_weather with arguments {"location":"Paris"}
Function call requested: get_weather with arguments {"location":"Paris"}
Function call requested: get_weather with arguments {"location":"Paris"}
Assistant: [ResponseOutputText(annotations=[], text='The weather in Paris is currently 25 °C.', type='output_text', logprobs=[])]
Function call requested: set_weather with arguments {"location":"Paris","temp":45}
Function call requested: set_weather with arguments {"location":"Paris","temp":45}
Function call requested: set_weather with arguments {"location":"Paris","temp":45}
Function call requested: set_weather with argumen

## MCP

MCP stands for Model-Context Protocol. It allows LLMs communicate
with different tools (like Qdrant). It's function calling, but 
one step further:

* A tool can export a list of functions it has
* When we include the tool to our Agent, we just need to include the link to the MCP server

## Q3. Install FastMCP

Let's install a library for MCP - [FastMCP](https://github.com/jlowin/fastmcp):

```bash
pip install fastmcp
```

What's the version of FastMCP you installed?

In [ ]:
#!uv add fastmcp

import fastmcp

# version of fastmcp
print(fastmcp.__version__)

2.10.6


## Q4. Simple MCP Server 

A simple MCP server from the documentation looks like that:

```python
# weather_server.py
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    mcp.run()
```

In our case, we need to write docstrings for our functions.

Let's ask ChatGPT for help:

```python
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'
```

Let's change the example for our case and run it

What do you see in the output?

Look for a string that matches this template:

```
Starting MCP server 'Demo 🚀' with transport '<TODO>'
```

What do you have instead of `<TODO>`? - stdio ✅

In [24]:
# start mcp server in weather.py
#!uv run weather.py

## Q5. Protocol

There are different ways to communicate with an MCP server.
Ours is currently running using standart input/output, which
means that the client write something to stdin and read the
answer using stdout.

Our weather server is currently running.

This is how we start communitcating with it:

- First, we send an initialization request -- this way, we register our client with the server:
    ```json
    {"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
    ```
    We should get back something like that, which is an aknowledgement of the request:
    ```json
    {"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.9.4"}}}
    ```
-  Next, we reply back, confirming the initialization:
    ```json
    {"jsonrpc": "2.0", "method": "notifications/initialized"}
    ```
    We don't expect to get anything in response
- Now we can ask for a list of available methods:
    ```json
    {"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
    ```
- Let's ask the temperature in Berlin:
    ```json
    {"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "<TODO>", "arguments": {<TODO>}}}
    ```
- What did you get in response?

In [59]:
# start fastmcp server
# uv run weather_server.py

In [ ]:
import subprocess

proc = subprocess.Popen(
    ["python", "weather_server.py"],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
)


def send_to_mcp(proc, request_json, expect_response=True):
    proc.stdin.write(request_json + "\n")
    proc.stdin.flush()
    if expect_response:
        return proc.stdout.readline().strip()


# 1. Initialize
init_request = '{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}'
print(send_to_mcp(proc, init_request, expect_response=True))

print("\n")

# 2. Confirm initialization (no response expected)
confirm_init = '{"jsonrpc": "2.0", "method": "notifications/initialized"}'
send_to_mcp(proc, confirm_init, expect_response=False)

print("\n")

# 3. List tools
list_tools = '{"jsonrpc": "2.0", "id": 2, "method": "tools/list", "params": {}}'
print(send_to_mcp(proc, list_tools, expect_response=True))

print("\n")

# 4. Call get_weather for Berlin
import json  # Add this at the top if not present

call_get_weather = json.dumps(
    {
        "jsonrpc": "2.0",
        "id": 3,
        "method": "tools/call",
        "params": {"name": "get_weather", "arguments": {"city": "Berlin"}},
    }
)
print(send_to_mcp(proc, call_get_weather, expect_response=True))

print("\n")

proc.terminate()

{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Weather 🚀","version":"1.12.0"}}}




{"jsonrpc":"2.0","id":2,"result":{"tools":[{"name":"get_weather","description":"Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.","inputSchema":{"properties":{"city":{"title":"City","type":"string"}},"required":["city"],"type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"number"}},"required":["result"],"title":"_WrappedResult","type":"object","x-fastmcp-wrap-result":true}},{"name":"set_weather","description":"Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\

## Q6. Client

We typically don't interact with the server by copy-pasting 
commands in the terminal.

In practice, we use an MCP Client. Let's implement it. 

FastMCP also supports MCP clients:

```python
from fastmcp import Client

async def main():
    async with Client(<TODO>) as mcp_client:
        # TODO
```

Use the client to get the list of available tools
of our script. How does the result look like?

If you're running this code in Jupyter, you need to pass
an instance of MCP server to the `Client`:

```python
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        # ....
```

If you run it in a script, you will need to use asyncio:

```python
import asyncio

async def main():
    async with Client("weather_server.py") as mcp_client:
        # ...

if __name__ == "__main__":
    test = asyncio.run(main())
```

Copy the output with the available tools when
filling in the homework form.

In [ ]:
import weather_server
from fastmcp import Client
import asyncio


async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print(tools)


await main()

[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Te